# **Implementation of Logistic Regression**
In this Python Notebook the implememtation of logistic regression is done

In [1]:
import numpy as np
import pandas as pd

### Loading the data

In [6]:
df= pd.read_csv("data/ex2data1.csv")
df

,x1,x2,y
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1
...,...,...,...
95,83.489163,48.380286,1
96,42.261701,87.103851,1
97,99.315009,68.775409,1
98,55.340018,64.931938,1


In [13]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',')
    X = data[:,:2]
    y = data[:,2]
    return X, y

In [14]:
X_train, y_train = load_data("data/ex2data1.txt")

### Implementing the sigmoid function

In [2]:
def sigmoid(z):
    """
    Compute the sigmoid activation function for a given input.

    Parameters:
    z (float or numpy.ndarray): The input value or array.

    Returns:
    float or numpy.ndarray: The sigmoid of the input value or each element of the input array.
    """

    g = 1 / (1 + np.exp(-z))
    return g


### Implementing the cost function

**Without regularization**

In [24]:
import numpy as np

def compute_cost(X, y, w, b):
    """
    Compute the logistic regression cost function for given parameters.

    Parameters:
    X (numpy.ndarray): Input feature matrix with shape (m, n), where m is the number of samples and n is the number of features.
    y (numpy.ndarray): Binary labels with shape (m,), where m is the number of samples.
    w (numpy.ndarray): Weight vector with shape (n,).
    b (float): Bias term.

    Returns:
    float: The computed cost value based on the logistic regression model's predictions and the provided parameters.
    """

    m, n = X.shape
    total_cost = 0

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)
        total_cost += y[i] * np.log(f_wb_i) + (1 - y[i]) * np.log(1 - f_wb_i)

    total_cost = -total_cost / m
    return total_cost

        

**With regularization**

In [30]:
def compute_cost_reg(X, y, w, b, lambda_=1):
    """
    Calculate the regularized logistic regression cost function with L2 regularization.

    This function computes the regularized logistic regression cost based on the provided input features, binary labels,
    weights, bias, and L2 regularization parameter.

    Parameters:
    X (numpy.ndarray): Input feature matrix with shape (m, n), where m is the number of samples and n is the number of features.
    y (numpy.ndarray): Binary labels with shape (m,), where m is the number of samples.
    w (numpy.ndarray): Weight vector with shape (n,).
    b (float): Bias term.
    lambda_ (float, optional): Regularization parameter. Default is 1.

    Returns:
    float: The computed regularized logistic regression cost value.
    """

    m, n = X.shape
    
    # Calculate the cost without regularization using the provided function
    cost_without_reg = compute_cost(X, y, w, b)
    
    # Calculate the sum of squared weights for regularization
    reg_cost = 0
    for i in range(n):
        reg_cost += w[i] ** 2
    
    # Apply L2 regularization term
    reg_cost = (lambda_ * reg_cost) / (2 * m)
    
    # Combine the original cost and the regularization term
    total_cost = cost_without_reg + reg_cost
    return total_cost


### Finding Gradient

**Without regularization**

In [48]:
def compute_gradient(X, y, w, b):
    """
    Compute the gradient of the logistic regression cost function with respect to the weights and bias.

    This function calculates the gradient of the logistic regression cost function with respect to the weights
    and bias terms. It is used in the gradient descent optimization algorithm to update the parameters.

    Parameters:
    X (numpy.ndarray): Input feature matrix with shape (m, n), where m is the number of samples and n is the number of features.
    y (numpy.ndarray): Binary labels with shape (m,), where m is the number of samples.
    w (numpy.ndarray): Weight vector with shape (n,).
    b (float): Bias term.

    Returns:
    numpy.ndarray, float: The gradient of the cost function with respect to weights (dj_dw) and bias (dj_db).
    """

    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.0
    
    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)
        err_i = f_wb_i - y[i]
        
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i, j]
        dj_db = dj_db + err_i
    
    dj_dw = dj_dw / m                                   
    dj_db = dj_db / m 
    
    return dj_dw, dj_db


**With regularizarion**

In [51]:
def compute_gradient_reg(X, y, w, b, lambda_):
    """
    Compute the gradient of the regularized logistic regression cost function with respect to the weights and bias.

    This function calculates the gradient of the regularized logistic regression cost function with respect to the weights
    and bias terms. L2 regularization is applied to the gradient of the weights.

    Parameters:
    X (numpy.ndarray): Input feature matrix with shape (m, n), where m is the number of samples and n is the number of features.
    y (numpy.ndarray): Binary labels with shape (m,), where m is the number of samples.
    w (numpy.ndarray): Weight vector with shape (n,).
    b (float): Bias term.
    lambda_ (float): Regularization parameter.

    Returns:
    float, numpy.ndarray: The gradient of the cost function with respect to bias (dj_db) and weights (dj_dw).
    """

    dj_dw, dj_db = compute_gradient(X, y, w, b)
    m, n = X.shape
    
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_ / m) * w[j]
    
    return dj_db, dj_dw


### Implementing Gradient Descent

In [56]:
def gradient_descnet(X,y,w_in,b_in,alpha,num_iters):
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
      
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history         #return final w,b and J history for graphing